In [124]:
from pytrends.request import TrendReq
import time
KEYWORD = 'bitcoin'
TODAY = time.strftime("%Y%m%d")
FOLDER='data'

# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

# Create payload and capture API tokens. Only needed for interest_over_time(), interest_by_region() & related_queries()
pytrend.build_payload(kw_list=[KEYWORD])

# Interest Over Time
interest_over_time_df = pytrend.interest_over_time()
interest_over_time_df = interest_over_time_df.drop('isPartial', 1)
interest_over_time_df = interest_over_time_df.rename(columns = {'bitcoin': 'trend', 'date': 'Date'})
print(interest_over_time_df.info())
interest_over_time_df.to_csv('%s/gtrends_%s_%s.csv' % (FOLDER, KEYWORD, TODAY))
interest_over_time_df.to_csv('%s/gtrends_%s_latest.csv' % (FOLDER, KEYWORD))
print(interest_over_time_df)

# Interest by Region
interest_by_region_df = pytrend.interest_by_region()
interest_over_time_df.to_csv('%s/gtrends_region_%s_%s.csv' % (FOLDER, KEYWORD, TODAY))
print(interest_by_region_df)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 260 entries, 2013-01-06 to 2017-12-24
Data columns (total 1 columns):
trend    260 non-null int64
dtypes: int64(1)
memory usage: 4.1 KB
None
            trend
date             
2013-01-06      1
2013-01-13      1
2013-01-20      1
2013-01-27      1
2013-02-03      1
2013-02-10      1
2013-02-17      1
2013-02-24      1
2013-03-03      1
2013-03-10      1
2013-03-17      2
2013-03-24      3
2013-03-31      6
2013-04-07     11
2013-04-14      6
2013-04-21      4
2013-04-28      3
2013-05-05      2
2013-05-12      2
2013-05-19      2
2013-05-26      2
2013-06-02      2
2013-06-09      2
2013-06-16      1
2013-06-23      2
2013-06-30      2
2013-07-07      2
2013-07-14      1
2013-07-21      1
2013-07-28      1
...           ...
2017-06-04     12
2017-06-11     13
2017-06-18     11
2017-06-25     11
2017-07-02      9
2017-07-09     10
2017-07-16     12
2017-07-23     11
2017-07-30     15
2017-08-06     14
2017-08-13     21
2017-08-20    

In [31]:
import json
import pandas

def readDfFromJson(fileName, field='Data'):
    with open(fileName) as data_file:    
        data = json.load(data_file)
    df = pandas.DataFrame(data[field])
    return df

In [129]:
from os import listdir
from os.path import isfile,join


# Read coin info
FOLDER = 'data'
SURFIX = '_USD_latest.json'
dfs = []
result = None

jFiles = [f for f in listdir(FOLDER) if isfile(join(FOLDER, f)) and f.endswith(SURFIX)]
#jFiles = ['BTC_USD_latest.json', 'DASH_USD_latest.json']
for jFile in jFiles:
    name = jFile.split(SURFIX)[0]
    df = readDfFromJson(join(FOLDER, jFile))
    df = df[['time', 'open', 'high', 'close', 'volumeto']]
    df = df.rename(columns = {'open': name + '_open', 'high': name + '_high', 
                              'close': name + '_clsoe', 'volumeto': name + '_volumn'})
    df['Date'] = pandas.to_datetime(df['time'], unit='s', errors='ignore')
    df = df.drop('time', 1)
    if (jFile == 'BTC_USD_latest.json'):
        result = df    
    else:
        dfs.append(df)

for df in dfs:
    result = pandas.merge(result, df, on='Date', how='left')

    
# csv info
FOLDER = 'data'
SURFIX = '_latest.csv'
dfs = []

jFiles = ['BTC_difficulty_latest.csv', 'BTC_wallet_user_latest.csv', 'BTC_transition_latest.csv', 
          'BTC_total_coins_latest.csv', 'gold_latest.csv', 'oil_latest.csv', 'dollar_latest.csv']
for jFile in jFiles:
    name = jFile.split(SURFIX)[0]
    df = pandas.read_csv(join(FOLDER, jFile), parse_dates = ['Date'])
    df = df.rename(columns = {'Value': name})
    dfs.append(df)

#nasdaq
df = pandas.read_csv(join(FOLDER, 'nasdaq_latest.csv'), parse_dates = ['Trade Date'], usecols=['Trade Date', 'Index Value'])
df = df.rename(columns = {'Trade Date':'Date', 'Index Value': 'nasdaq'})    
dfs.append(df)

#google trend
df = pandas.read_csv(join(FOLDER, 'gtrends_bitcoin_latest.csv'), parse_dates = ['date'])
df = df.rename(columns = {'date':'Date'})    
dfs.append(df)

#merge
for df in dfs:
    result = pandas.merge(result, df, on='Date', how='left')

print(result.columns)
#result = result.drop(result.columns[0], axis=1)
result.to_csv('data/result.csv')
result.head(4)



Index(['BTC_open', 'BTC_high', 'BTC_clsoe', 'BTC_volumn', 'Date', 'DASH_open',
       'DASH_high', 'DASH_clsoe', 'DASH_volumn', 'ETH_open', 'ETH_high',
       'ETH_clsoe', 'ETH_volumn', 'GNT_open', 'GNT_high', 'GNT_clsoe',
       'GNT_volumn', 'IOT_open', 'IOT_high', 'IOT_clsoe', 'IOT_volumn',
       'LTC_open', 'LTC_high', 'LTC_clsoe', 'LTC_volumn', 'XRP_open',
       'XRP_high', 'XRP_clsoe', 'XRP_volumn', 'BTC_difficulty',
       'BTC_wallet_user', 'BTC_transition', 'BTC_total_coins', 'gold', 'oil',
       'dollar', 'nasdaq', 'trend'],
      dtype='object')


,BTC_open,BTC_high,BTC_clsoe,BTC_volumn,Date,DASH_open,DASH_high,DASH_clsoe,DASH_volumn,ETH_open,...,XRP_volumn,BTC_difficulty,BTC_wallet_user,BTC_transition,BTC_total_coins,gold,oil,dollar,nasdaq,trend
0,0.04951,0.04951,0.04951,0.9902,2010-07-17,NaN,NaN,NaN,NaN,NaN,...,NaN,181.543289,2.0,462.0,3436900.0,NaN,NaN,NaN,NaN,NaN
1,0.04951,0.08585,0.08584,5.0900,2010-07-18,NaN,NaN,NaN,NaN,NaN,...,NaN,181.543289,2.0,427.0,3445850.0,NaN,NaN,NaN,NaN,NaN
2,0.08584,0.09307,0.08080,49.6600,2010-07-19,NaN,NaN,NaN,NaN,NaN,...,NaN,181.543289,2.0,487.0,3454050.0,1181.0,NaN,76.6584,2198.23,NaN
3,0.08080,0.08181,0.07474,20.5900,2010-07-20,NaN,NaN,NaN,NaN,NaN,...,NaN,181.543289,2.0,627.0,3462950.0,1183.0,NaN,76.7410,2222.49,NaN
